# Introduction to GeoPandas for Working with Geospatial Vector Data (i.e., shapes: points, lines, and polygons)

# I. Importing Necessary Packages

In [ ]:
import geopandas as gpd
# import shapely
# import fiona
import pyogrio

# to avoid the GDAL error you need to activate the environment before launching jupyter

# II. Why use Python GeoPandas for geospatial data analysis as opposed to ArcGIS or other GIS software?

If most or all of the data you work with are vector-based spatial data formats such as shapefiles, GeoJSON, or geodatabases, using GIS software will be more desirable and efficient!

On the other hand, if most of the data you work with are not in these formats and your workflow is primarily in Python, GeoPandas may be the tool for you. For example, for a geoscientist (earth, environmental, climate sciences) who works most often with large multi-dimensional data arrays (time,level,lat,lon) in formats like .nc, .tif, .grib, .hdf, .csv, Python will be a much more efficient and reproducible tool for scientific analysis. For this use case, packages like GeoPandas make working with the occasional shapefile or geodatabase very easy within your existing Python workflow instead of having to use a separate GIS tool for part of your analysis.

As we proceed through this notebook, we'll be looking at just of few of Python's geospatial analysis capabilities, mainly in the context that use of these tools is likely a minor part of a larger analysis using Python.

# III. Introduction to GeoPandas Data Structures

GeoPandas extends the Pandas package by adding support for geospatial vector data (i.e., shapes: points, lines, polygons). Under the hood, GeoPandas uses geometric operations from another Python package called [Shapely](https://shapely.readthedocs.io/en/stable/manual.html).

The core data structure in GeoPandas is the *GeoDataFrame*, which is similar to a Pandas DataFrame made up of an index and collection of *Series* (columns) as well as a special column named "geometry". The geometery column is a *GeoSeries* which is a special structure that can handle shapes (or "geometries") like points, lines, and polygons. Each shape object in a GeoPandas GeoSeries is a Shapely geometry object. A GeoDataFrame can contain as many Series and GeoSeries as you want.        

Below is a schematic of what a GeoPandas GeoDataFrame looks like, where the green boxes indicate the index, yellow boxes indicate data (with column names) in Series structures and pink boxes show a single GeoSeries structure which would have the column name "geometry".


<img src="images/geodataframe.svg" alt="schematic of a dataframe" width="700"/> 
         
(Image Source: [GeoPandas Docs Getting Started Tutorial](https://geopandas.org/en/stable/getting_started/introduction.html#Concepts)

# IV. Loading a ShapeFile into a GeoDataFrame

Most of the time you'll probably start a geospatial analysis with a shapefile or geodatabase, both of which are comprised of multiple components. Three components are mandatory for shapefiles: 
- a main file that contains the feature geometry (.shp),
- an index file that stores the index of the feature geometry (.shx), and
- a dBASE table (.dbf) that stores the attribute information of features.

The shapefiles we'll load contain the boundaries for a few US states and the centerlines for some large rivers. The files we're using contain subsets of data from larger files that are publicly available from the US Census Bureau and Natural Earth data project. If you're interested, the full data files are [cb_2023_us_state_500k.shp from the Census Bureau cartographic boundary files](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html) and [ne_10m_rivers_lake_centerlines.shp, ne_10m_rivers_north_america.shp from the Natural Earth data project](https://www.naturalearthdata.com/downloads/10m-physical-vectors/10m-rivers-lake-centerlines/).

```geopandas.read_file()``` can read almost any vector-based spatial data format.

In [ ]:
# load state boundaries from shapefile
gdf_states = gpd.read_file('data/state_boundaries/MS_LA_AR_AL_TN_cb_2023_us_state_500km.shp')
gdf_states

This is a GeoPandas GeoDataFrame and as you can see, it looks a lot like a Pandas DataFrame. Each row is a data entry for a different state. Notice the rightmost column named "geometry". It contains geospatial shapes that represent the boundaries of each state. 

Let's look at the data type of each column

In [ ]:
gdf_states.dtypes

You should remember seeing most of these data types from the Pandas notebook, but GeoPandas adds a new data type "geometry". This is indicating that the column named "geometry" is a GeoSeries that contains geospatial shapes.

Since there is only one column of type geometry, GeoPandas makes that column the "active geometry column." You can have more than one GeoSeries column that contain shapes, but only one active at a time. This doesn't really come into  play until it's time to create figures. 

Look onces again at the geometry column of the GeoDataFrame. Notice the values inside the geospatial shapes. What are those numbers? Each set of numbers (separated by commas) is a x,y node (point) that make up the shape. Large numbers, like we have, usually indicate that the data is in a coordinate reference system that has units of meters (as opposed to degrees latitude and longitude). We'll talk about coordinate reference systems next.

# V. Coordinate Reference Systems (CRS)

A CRS is a framework used to locate and map geographic features on the Earth's surface. The key components of a CRS are:

- **Coordinate System:** Defines the grid used to specify locations on the Earth, including the axes (like latitude/longitude or X/Y) and distance units of measurement (degrees or meters, for example). 

- **Datum:** The modeled version of the shape of the earth (usually represented by an ellipsoid). The datum also defines the origin (or reference point) of the coordinate system.

- **Projection:** A mathematical transformation that translates the spherical coordinates on the Earth's surface onto a flat surface like a map. No projection is perfect. Each will have different distortions which can affect area, shape, distance, and/or direction. Different projections are used based on the area of interest and the purpose of the map. Some coordinate reference systems do not have a projection, meaning their coordinate system and datum define locations on a 3-dimensional Earth rather than a 2-dimensional map. 


As mentioned above, there are actually two types of CRS's: geographic coordinate reference systems and projected coordinate reference systems.

- **Geographic coordinate reference systems** are used to locate places on a 3-dimensional Earth surface based on two values, longitude and latitude, usually with units of decimal degrees. Units of angular distances are not linear due to the polar convergence of longitudes and the slight bulge of the earth at the equator. Therefore, a geographic CRS is not suitable to calculate or compare distances between locations. A common geographic CRS would be EPSG:4326, which uses a latitude and longitude coordinate system and the WGS84 (an ellipsoid) datum to represent locations on the whole Earth. Confusingly, sometimes this CRS is referred to as the WGS84 CRS. I like to use the EPSG codes when discussing CRS's because they are unique identifiers that can eliminate a lot of confusion. EPSG:4326 is the default CRS used in many GPS systems and is also used by Google Earth.
- **Projected coordinate reference systems** are based on a Cartesian coordinate system on a flat surface. Map projections are used to convert the 3D surface of the Earth into x and y coordinates of the projected CRS. A common projected CRS that you may encounter is EPSG:3857, also known as the Web Mercator projection. This CRS is used for many web mapping applications include Google Maps and Apple Map.


Your choice of CRS will depend on how big your area of interest is, the region of the globe where it is located, and what you want to calculate. Projections that preserve shape are called conformal; those that preserve area are called equal-area; those that preserve distance are called equidistant; and those that preserve direction are called azimuthal. These words might assist you in finding a projected CRS that preserves that geospatial properties most important to your analysis. Also, keep in mind that you can "reproject" from one CRS to another to increase the accuracy of calculations that require preservation of different geospatial properties (e.g. area vs distance) .

Let's see what CRS our state boundaries use:

In [ ]:
gdf_states.crs

The CRS of this data is EPSG:5070, which uses the NAD83 datum and an X/Y Cartesian coordinate system with units in meters. The information GeoPandas gives us let's us know that this CRS is equal-area and is appropriate for areas of interest in the contiguous United States (CONUS).

Now, let's plot our state shapes.

In [ ]:
# plot geospatial data from our GeoDataFrame
gdf_states.plot()

Notice the units of the axes. These are the same units as described in the CRS, Cartesian X/Y in meters.

Also notice how we didn't specify which column of the ```gdf_states``` GeoDataFrame to plot. GeoPandas automatically plots the column named 'geometry'. If we had multiple GeoSeries columns (containing geospatial shapes) we would need to specify the column name if we wanted to override the default plotting of the geometery column.

What if we wanted to plot our data such that the axes show latitude and longitude instead of meters? We could "reproject" to the geographic CRS EPSG:4326 for the plot like this: 

In [ ]:
# plot geospatial data from our GeoDataFrame in a new CRS
gdf_states.to_crs('EPSG:4326').plot()

Note that we didn't overwrite our GeoDataFrame with a new CRS here, we are simply reprojecting for the plot and not saving the result. You can double check this if you want by looking at the output of ```gdf_states.crs``` again, as we did earlier. We'll see how to reproject a GeoDataFrame "in place" (overwriting the data) a little later.

# VI. General Methods and Attributes of GeoSeries

There is a lot of information you can pull out of the shape objects in a GeoSeries. Let's look at a few of the available options that may be useful. These methods work on the entire GeoSeries (all the shapes at once) and some of them also work on individual shapes from the GeoSeries. We'll only cover a few here.

First, let's look at a single shape object in the GeoSeries. We'll grab the Mississippi multipolygon and show that it is a Shapely geometry object.

In [ ]:
# visual of a single shape in the GeoSeries
MS = gdf_states.geometry.loc[3]
MS

It should make sense now why this shape is a group of polygons called a multipolygon- all the little barrier islands!

In [ ]:
# showing this is a shapely geometry object
type(MS)

## GeoSeries.boundary

Gives you just the shape outline

In [ ]:
MS.boundary

## GeoSeries.area

In [ ]:
MS.area

What units is this? It is based on the CRS units of meters, so the area is in square meters.

In [ ]:
# areas for all the states is square km
gdf_states.area /1E6

In [ ]:
# save the result to a new column in the GeoDataFrame
gdf_states['area_km2'] = gdf_states.area /1E6
gdf_states

## GeoSeries.bounds

Gives you the minx, miny, maxx, maxy bounds of the object in CRS units (here, meters)

In [ ]:
MS.bounds

Let's see how ```.bounds``` works on the entire GeoSeries.

In [ ]:
gdf_states.bounds

Let's look at some more methods that work on the full GeoSeries of shape objects

## GeoSeries.total_bounds

Gives you a single minx, miny, maxx, maxy bound of all the shapes combined 

In [ ]:
gdf_states.total_bounds

## GeoSeries.count_geometries()

For multi-geometries, this counts how many shapes are with the shape (e.g. polygons in a multipolygon). This will return a value of 1 for non multi-geometries.

In [ ]:
gdf_states.count_geometries()

## GeoSeries.count_coordinates()

Gives you the number of nodes (points) that make up each shape

In [ ]:
gdf_states.count_coordinates()

## GeoSeries.get_coordinates()

Pulls the x and y value of each point in each shape and puts them into separate x/y columns

In [ ]:
gdf_states.get_coordinates()

You can find the full list of [GeoSeries general methods and attributes in the GeoPandas Documentation](https://geopandas.org/en/stable/docs/reference/geoseries.html#general-methods-and-attributes).

# VII. Showing Multiple GeoSeries on a Single Figure

Let's read another shapefile containing the centerlines of some large rivers and plot them along with the state boundaries in a single figure.

In [ ]:
# get river data from shapefile into a GeoDataFrame
gdf_rivers = gpd.read_file("data/natural_earth_rivers/ms_clipped_major_rivers/ms_clipped_major_rivers.shp")
gdf_rivers

Unlike in GIS, there is no "on the fly" reprojection with GeoPandas or any other Python package. You need to be aware of the CRS's that your different datasets are using and make sure they match in order for the data to overlay correctly in a figure.

Let's check the CRS of ```gdf_rivers```

In [ ]:
# check the crs
gdf_rivers.crs

Excellent, this data is also in EPSG:5070, meaning ```gdf_states``` and ```gdf_rivers``` should overlay correctly if we plot them together.

To plot more than one GeoSeries you must first create an 'Axes' object which is like a base figure, then plot you can plot more GeoSeries on top of it like this:

In [ ]:
# create the base figure called ax and plot the states on it
base = gdf_states.plot(facecolor='none',edgecolor='black',lw=0.5)

# add the rivers on top of the base figure
gdf_rivers.plot(ax=base)

# VIII. Loading Data from a Geodatabase into a GeoDataFrame

The geodatabase (.gdb) format is proprietary to ESRI, the developer of ArcGIS, however, GeoPandas ```.read_file()```is able to load data from a geodatabase. 

Let's load superfund site data from a geodatabase. Superfund sites are locations in the US where hazardous substances, pollutants, or contaminants are known or suspected to be released. The US Environmental Protection Agency (EPA) maintains the National Priorities List (NPL) of superfund sites. The NPL contains data that documents these sites and their clean up. 

The data file we will work with has been subset from a much larger publicly available data file by filtering for selected data columns for sites in Mississippi that include geospatial location. [The larger data file can be obtained from data.gov](https://catalog.data.gov/dataset/npl-superfund-site-boundaries-epa9), if you are interested.   

In [ ]:
gdf_sites=gpd.read_file('data/EPA_superfund_boundaries/MS_EPA_NPL_Site_Boundaries.gdb')
gdf_sites

This geodatabase is very simple, as it only contains one GIS data layer of multipolygon objects. This will rarely be the case when you are working with geodatabases. Before we move on, let's look at the tools you can use to see the names of the layers in a geodatabase and how you would read in a particular data layer if you had more than one. We need to import an additional package for this called pyogrio (we've already imported it in the beginning of our notebook).

In [ ]:
# use the pyogrio package to list all the layer names in a geodatabase
pyogrio.list_layers('data/EPA_superfund_boundaries/MS_EPA_NPL_Site_Boundaries.gdb')

This output is indicating that all the data in our geodatabase is stored on a single GIS data layer called 'MS_EPA_NPL_Site_Boundaries' and the layer's contents are multipolygons. If you are familiar with GIS, you know that there could be many more layer names in this list. If we had more layers the ```pyogrio.list_layers()``` output might look like this:

```array([['MS_EPA_NPL_Site_Boundaries', 'MultiPolygon'],['some_other_layer_name', 'layer_geometry_type'],['some_other_layer_name', 'layer_geometry_type']], dtype=object)```



To load data from a specific layer into a GeoDataFrame we simply need to add the layer parameter when we read the geodatabase, like this:

In [ ]:
gdf_sites=gpd.read_file('data/EPA_superfund_boundaries/MS_EPA_NPL_Site_Boundaries.gdb', layer='MS_EPA_NPL_Site_Boundaries')
gdf_sites

Moving on, let's check what CRS this data uses.

In [ ]:
gdf_sites.crs

It's in a different CRS! If we want to look at the locations of the superfund sites in relation to major rivers, we'll have to reproject this data to the CRS of our other GeoSeries, which is EPSG:5070.

Let's reproject this data to EPSG:5070, overwriting the current CRS and geometry column of ```gdf_sites```. To do this we use ```.to_crs()``` with the parameter ```inplace``` set to True. 

In [ ]:
gdf_sites.to_crs('epsg:5070',inplace=True)
gdf_sites

Notice the nodes of each multipolygon have changed from units of decimal degrees to units of meters. Let's double check the new CRS info.

In [ ]:
gdf_sites.crs

Great! All our GeoSeries now have the same EPSG:5070 CRS. Let's plot the state boundaries, rivers, and superfund sites on a single figure.

In [ ]:
base = gdf_states.plot(facecolor='none',edgecolor='black',lw=0.5)
gdf_rivers.plot(ax=base,color='blue')
gdf_sites.plot(ax=base,color='orange')

What happened, why can't we see the orange sites??

If we were using a fancier package for plotting (like folium) we could zoom in and see them, but with this simple static plotting the problem is that the sites are too small to see at this scale.  We can visualize the sites at this scale by plotting their centroid points, which will automatically scale to be visible.

# IX. Geometric Constructive Functions

https://geopandas.org/en/stable/docs/user_guide/geometric_manipulations.html

## Centroid 

In [ ]:
# create a new GeoSeries in the GeoDataFrame that contains the centroid (point) of each multipolygon
gdf_sites['centroid']=gdf_sites.geometry.centroid
gdf_sites

See how we can have more than one column that is a GeoSeries? The two columns of geospatial shapes are both GeoSeries. If we print the data types of all the columns we should see that we have two GeoSeries now. 

In [ ]:
# check out the data types
gdf_sites.dtypes

As mentioned earlier, the default behavior when plotting a GeoSeries in a GeoDataFrame is to plot whatever shapes are in the column named geometry. If you want to plot a different GeoSeries, you have to specify the column name like we do below with ```gdf_sites.centroid.plot()```

Trying our plot of states, rivers, and superfund sites again.

In [ ]:
base = gdf_states.plot(facecolor='none',edgecolor='black')
gdf_rivers.plot(ax=base,color='blue')
gdf_sites.centroid.plot(ax=base,color='orange')

Ooof, nearly all the superfund sites are near major rivers or on the coast. That can't be good for water quality! We'll use some additonal functions to see how close these sites are to the major rivers in our data.

## Buffer

First, let's make some simple buffers around our rivers. The [GeoPandas API reference](https://geopandas.org/en/stable/docs/reference.html) is where you should go to find out about all the parameters that you can input to a GeoPandas function. Here's the direct link to the page for [.buffer()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html).

In [ ]:
# 100 meter buffer
gdf_rivers["buffer_100m"] = gdf_rivers.buffer(100)

# 1000 meter buffer
gdf_rivers["buffer_1000m"] = gdf_rivers.buffer(1000)

# plot
base = gdf_states.plot(facecolor='none',edgecolor='black',figsize=(12,6)) # larger figure than default
gdf_sites.centroid.plot(ax=base,color='orange')
gdf_rivers.geometry.plot(ax=base,color='blue')
gdf_rivers.buffer_1000m.plot(ax=base,color='brown',alpha=0.25)

# change x and y extents
base.set_xlim(0.575E6,0.75E6)
base.set_ylim(.8E6,.9E6)

Notice the few things we added to our plotting code. We've changed the figure size with the ```figsize``` parameter and we've changed the x and y extents of the plot with the functions ```.set_xlim()```, ```.set_ylim()```. These changes result in a larger figure that is zoomed in on coastal Mississippi.

## Convex Hull
For a geometry with at least 3 points, the convex hull is the smallest convex Polygon containing all the points of the geometry. We'll create a convex hull polygon for each state and then plot the Mississippi state multipolygon and convex hull polygon on the same figure.

In [ ]:
gdf_states['convex_hulls']=gdf_states.convex_hull
gdf_states

In [ ]:
base = gdf_states.loc[gdf_states.STUSPS=='MS'].plot(facecolor='none',edgecolor='black')
gdf_states.loc[gdf_states.STUSPS=='MS'].convex_hulls.plot(ax=base,facecolor='yellow',edgecolor='orange',alpha=0.5)

## Envelope

An envelope is the smallest rectangular polygon (with sides parallel to the coordinate axes) that contains a geometry.

In [ ]:
gdf_states['envelopes']=gdf_states.envelope
gdf_states

In [ ]:
base = gdf_states.plot(facecolor='none',edgecolor='black')
gdf_states.envelopes.plot(ax=base,alpha=0.5)

## Union All

Returns a single shape (not a GeoSeries!) containing the union of all geometries in the GeoSeries.

In [ ]:
gdf_states.union_all()

In [ ]:
type(gdf_states.union_all())

# Binary (True/False) Functions

These are functions that operate on a GeoSeries and return the result True or False for each shape in the GeoSeries

## Intersects

In [ ]:
pearl_river = gdf_rivers.loc[1,"geometry"]

In [ ]:
base = gdf_states.plot(facecolor='none',edgecolor='black')
gdf_rivers.loc[gdf_rivers.name=='Pearl'].plot(ax=base)

In [ ]:
gdf_states['intersects_pearl']=gdf_states.intersects(pearl_river)
gdf_states

## Contains and Within

Find whether one shape contains another or similarly, whether one shape is within another. E.g., polygon A *contains* point B, point B is *within* polygon A. It's just two ways of finding the same information. 



In [ ]:
# check if each state polygon contains a single superfund site
one_site = gdf_sites.loc[0,"geometry"]

gdf_states.contains(one_site)

In [ ]:
# check if 1 state contain any of the superfund sites
one_state = gdf_states.geometry.iloc[0]  # this is Alabama

one_state.contains(gdf_sites.geometry).any()

In [ ]:
# get at the same information using within instead of contains

# check if any of the superfund sites are within Alabama
gdf_sites.geometry.within(one_state).any()

Now for a little added complexity! What if we want to check each state in ```gdf_states``` to see whether they contain any of the superfund sites in our dataset?

We will have to write a function for that and then use ```.apply()```, which will apply our function to each row of a GeoDataFrame. In this case, we'll apply our function to the rows of ```gdf_states```, checking whether each state shape contains any of the superfund sites. We already know that our superfund site data has been subset to Mississippi, so we expect a result of True for Mississippi and False for all other states. Here's how we can do it:

In [ ]:
# check if any sites are within each state

def shape_contains_shape(row,sites):
    shape_object = row.geometry  # a single shapely object (the geometry column from each row)
    return shape_object.contains(sites.geometry).any() # this part is the same as we saw above

# apply the function to each row of gdf_states
# and save the output to a new column
# axis=1 indicates that we want to apply the function to each row of gdf_states
gdf_states['has_sites'] = gdf_states.apply(shape_contains_shape,axis=1,args=(gdf_sites,))
gdf_states

```.apply()``` is like a loop, applying our function to 1 row of ```gdf_states``` at a time, moving row by row until the end of all the rows in the GeoDataFrame, then returning the results.

Let's check if any of the superfund sites are within the 1000 meter river buffers.

In [ ]:
# check if any site centroids are within the 1000 meter river buffers

# this time we'll use within
def shape_within_shape(row, col1, gdf, col2):
    shape1 = row[col1]  # a single shapely object (from a particular column of each row)
    return shape1.within(gdf[col2]).any()

# apply the function to each row of gdf_sites and save the output to a new column
gdf_sites['centroid_in_buff1000'] = gdf_sites.apply(shape_within_shape, axis=1, args=('centroid',gdf_rivers,'buffer_1000m'))
gdf_sites

If we scroll to the right to see the new column ```centroid_in_buff1000```, we can see that even though the superfund centroid points we plotted looked very close to our major rivers, none of the centroids are within the 1000 meter river buffers. Parts of the superfund site polygons could be within the buffers though, we'll check that later. 

There are quite a few more GeoPandas functions like this called "Binary Predicates" and they can be found in the [GeoPandas API reference here](https://geopandas.org/en/stable/docs/reference/geoseries.html#binary-predicates)

# Clipping

# Clip Shapes with Another Shape

Clip the rivers to the rectangular envelope for Mississippi.

First, let's work out the syntax for selecting a single Shapely object from a GeoSeries. This means selecting the polygon shape from the envelopes column of the gdf_states GeoDataFrame. The selection, if done correctly, should result in a single shape rendered to the screen, not a column or row of data:

In [ ]:
# select a single Shapely polygon (Mississippi envelope)
gdf_states.loc[gdf_states.STUSPS=='MS']['envelopes'].iloc[0]

In [ ]:
# clip rivers with Mississippi envelope
clipped_rivers = gpd.clip(gdf_rivers, gdf_states.loc[gdf_states.STUSPS=='MS']['envelopes'].iloc[0])

# plot states, Mississippi envelope, and clipped rivers
base = gdf_states.plot(facecolor='none',edgecolor='black')
gdf_states.loc[gdf_states.STUSPS=='MS']['envelopes'].boundary.plot(ax=base,color='red')
clipped_rivers.plot(ax=base,color='blue')

You can also clip raster data (data with pixels or on a grid) with GeoPandas, but for that we will need an additional Python packages. We'll learn how to do that in a subsequent lesson.

# Distance

In [ ]:
# dropping more columns that we don't need before we do the calculation
# gdf_sites.drop(columns=['EPA_PROGRAM','EPA_ID','URL_ALIAS_TXT','CITY_NAME','COUNTY','STATE_CODE','ZIP_CODE'],inplace=True)
gdf_rivers.drop(columns=['scalerank','featurecla','rivernum','ne_id','wikidataid'],inplace=True)

In [ ]:
sites_with_closest_river_info = gpd.sjoin_nearest(gdf_sites, gdf_rivers, how='left', rsuffix='rivers',distance_col='min_distance', exclusive=True)
sites_with_closest_river_info

The results in the min_distance column indicate there are 3 sites that are at least partially in a 1000m river buffer.

Look at index 12. The closest part of the Rockwell International Wheel & Trim superfund polygon is only 528 meters from the Yalobusha River centerline. Let's make a zoomed in plot of this site's polygon and centroid, the Yalobusha River, and the 1000 meter buffer. 

In [ ]:
# plot Rockwell International polygon and centroid, Yalobusha River, and 1000 meter buffer

base = gdf_sites.loc[gdf_sites.SITE_NAME.str.contains('ROCKWELL')].boundary.plot(color='red',figsize=(8,8))
gdf_sites.loc[gdf_sites.SITE_NAME.str.contains('ROCKWELL')].centroid.plot(ax=base,color='orange')
gdf_rivers.loc[gdf_rivers.name=='Yalobusha'].plot(ax=base,color='blue')
gdf_rivers.loc[gdf_rivers.name=='Yalobusha']['buffer_1000m'].plot(ax=base,color='brown',alpha=0.5)

# zoom in
base.set_ylim(1.209E6,1.213E6)
base.set_xlim(.568E6,.571E6)

Look at that! The centroid is not in the buffer, but part of the site polygon is. If we wanted to create a column telling us which site polygons lie partially within the river buffers we could use ```.intersects()``` for that.

## Exercise

Create a new column in ```gdf_sites``` called ```intersects_buff1000``` to test whether each site polygon lies at least partially within any of the river buffers (you can use ```.intersects()``` for this). You will need to write a new function called ```shape_intersects_shape``` and apply it to the ```gdf_sites``` GeoDataFrame. Use the ```shape_within_shape``` function that we made earlier as an example.

In [ ]:
# add you code here

def shape_intersects_shape(row, col1, gdf, col2):
    shape1 = row[col1]  # a single shapely object (from a particular column of each row)
    return shape1.intersects(gdf[col2]).any()

# apply the function to each row of gdf_sites and save the output to a new column
gdf_sites['intersects_buff1000'] = gdf_sites.apply(shape_intersects_shape, axis=1, args=('geometry',gdf_rivers,'buffer_1000m'))
gdf_sites

Use ```.sum()``` on the column ```intersects_buff1000``` to show programatically how many sites are at least partially in a 1000 meter river buffer.

In [ ]:
# add your code here
gdf_sites.intersects_buff1000.sum()

# Aggregation with .dissolve()

## Aggregating Shapes

First lets aggregate rivers with the same name together. We've got repeated river names in ```gdf_rivers```. How do we know this? We can compare the number of rows in ```gdf_rivers``` to the number of unique river names:

In [ ]:
print(f'There are {gdf_rivers.shape[0]} rows in gdf_rivers, but only {len(gdf_rivers.name.unique())} unique river names')

In [ ]:
# test = gdf_rivers[['name','geometry']].dissolve(by='name')
# test
gdf_rivers = gdf_rivers[['name','geometry']].dissolve(by='name',as_index=False,sort=False)

In [ ]:
gdf_rivers["buffer_1000m"] = gdf_rivers.buffer(1000)
gdf_rivers

In [ ]:
# plot, rebuffer?

## aggregate data by shape
I may need different data for this






## intersection
## union


# Binary (True/False) Functions
## intersects
## overlaps
## contains

# Geometric Overlay Operations 
## .clip()
## .overlay()

# Other Useful Things 
## distance


# Aggregation
## dissolve

# .explore()


In [ ]:
ax = gdf_states.geometry.boundary.plot(color='black',lw=0.5)#facecolor='none',edgecolor='black')


In [ ]:
import openpyxl
import pandas as pd

In [ ]:
df_toxic = pd.read_csv('data/epa_brownfields_ms/STATE_SINGLE_MS.csv')
df_toxic

In [ ]:
df_toxic.dropna(subset=['LATITUDE83','LONGITUDE83'],inplace=True)
df_toxic

In [ ]:
df_toxic.columns

In [ ]:
df_toxic.dropna(subset=['HDATUM_DESC'],inplace=True)
df_toxic

In [ ]:
df_toxic.HDATUM_DESC.unique()

In [ ]:
df_toxic.SITE_TYPE_NAME.unique()

In [ ]:
df_toxic.INTEREST_TYPES.unique()

In [ ]:
df_toxic.INTEREST_TYPES.dtype

In [ ]:
df_toxic.INTEREST_TYPES.fillna('',inplace=True)

In [ ]:
df_toxic[df_toxic.INTEREST_TYPES.isin(['TRI REPORTER','TSCA SUBMITTER'])]
# df_toxic.query("INTEREST_TYPES.str.contains('TRI')")

In [ ]:
df_toxic.drop(columns=['LOCATION_ADDRESS', 'SUPPLEMENTAL_LOCATION', 'CITY_NAME','FIPS_CODE', 'STATE_CODE',
                       'COUNTRY_NAME','EPA_REGION_CODE','CREATE_DATE', 'UPDATE_DATE', 'US_MEXICO_BORDER_IND',
                       'CONVEYOR','COLLECT_DESC', 'ACCURACY_VALUE','SOURCE_DESC','PGM_SYS_ACRNMS',
                       'NAICS_CODES', 'NAICS_CODE_DESCRIPTIONS','SIC_CODES','SIC_CODE_DESCRIPTIONS',
                       'FEDERAL_FACILITY_CODE', 'FEDERAL_AGENCY_NAME', 'TRIBAL_LAND_CODE','TRIBAL_LAND_NAME',
                       'LOCATION_DESCRIPTION'],inplace=True)
df_toxic

In [ ]:
df_toxic = df_toxic[df_toxic.INTEREST_TYPES.isin(['TRI REPORTER','TSCA SUBMITTER'])]
df_toxic

In [ ]:
# import fiona
# layers=fiona.listlayers("data/acres_frs/ACRES_FRS.kml")
# layers

In [ ]:
# gdf_toxic = gpd.read_file("data/acres_frs/ACRES_FRS.kml", driver="KML")
# gdf_toxic

In [ ]:
study_area = gdf_states.geometry.union_all()
study_area

In [ ]:
gdf_rivers_ms = gpd.clip(gdf_rivers,ms)
gdf_rivers_ms

In [ ]:
# gdf_ms = gpd.read_file("data/tl_2016_28_cousub/tl_2016_28_cousub.shp")
# gdf_ms.head()

In [ ]:
gdf_ms.plot()

In [ ]:
gdf_ms.crs

In [ ]:
gdf_ms = gdf_ms.to_crs('epsg:5070')
gdf_ms.crs

In [ ]:
ms = gdf_ms.geometry.union_all()
ms

In [ ]:
gdf_rivers = gdf_rivers.to_crs('epsg:5070')
gdf_rivers.crs

In [ ]:
gdf_rivers_ms = gpd.clip(gdf_rivers,ms)
gdf_rivers_ms

In [ ]:
gdf_rivers_ms.plot()